In [1]:
import os
import re
import csv
import datetime
import reportlab

from datetime import datetime
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import mm
from reportlab.pdfbase.pdfmetrics import stringWidth

In [162]:
# params
# --------------------------------------
WIDTH = 612.0 # default pagesize letter
HEIGHT = 792.0 # default pagesize letter
MARGIN = 36.0

PUZZLE_TOP_Y = 550 # distance from bottom of page that boxes/rect start --> should depend on other vars?
LINE_SPACING = 25
FOOTER_SPACING = LINE_SPACING # distance from bottom of last box to top of clue, vertically

BOX_LEN = 25 # space between boxes = BOX_LEN
TRIANGLE_HEIGHT = 25

TITLE_FONT_SIZE = 25
DESCRIPTION_FONT_SIZE = 14
LETTER_FONT_SIZE = 18
BODY_FONT_SIZE = 12
BODY_FONT = "Helvetica"
BODY_FONT_BOLD = "Helvetica-Bold"
BODY_FONT_OBLIQUE = "Helvetica-Oblique"
BODY_FONT_BOLD_OBLIQUE = "Helvetica-BoldOblique"
FOOTER_FONT_SIZE = 13

CIRCLE_RADIUS = 22
CIRCLE_SPACE = 5 # horizontal space between circles
CIRCLE_Y = PUZZLE_TOP_Y + TRIANGLE_HEIGHT + BOX_LEN + CIRCLE_RADIUS + 20 # 20 is tunable

LOG_FILENAME = "log.csv"
FOLDER = "puzzles/"

PATTERN = '[^A-Za-z ]'

CHAR_TO_NUM = {'A': 1, 'B': 1, 'C': 1, 
               'D': 2, 'E': 2, 'F': 2,
               'G': 3, 'H': 3, 'I': 3,
               'J': 4, 'K': 4, 'L': 4,
               'M': 5, 'N': 5, 'O': 5,
               'P': 6, 'Q': 6, 'R': 6,
               'S': 7, 'T': 7, 'U': 7,
               'V': 8, 'W': 8,
               'X': 9, 'Y': 9, 'Z': 9}

CIRCLES = ['ABC','DEF','GHI','JKL','MNO','PQR','STU','VW','XYZ']

In [163]:
# useful definitions
#------------------------------------
x_l, x_h = MARGIN, WIDTH - MARGIN
y_l, y_h = MARGIN, HEIGHT - MARGIN
x_center, y_center = (x_l + x_h)/2, (y_l + y_h)/2
ch_per_line = int((WIDTH - (2 * MARGIN)) // BOX_LEN)
box_vertical_spacing = TRIANGLE_HEIGHT + BOX_LEN + LINE_SPACING

In [164]:
def separate_lines(proverb, n):
    line_intervals = [] # intervals of the form (start, end) for each generated line
    cur_line_interval = [0, None] 
    cur_len = 0 # length of the current line
    prev_word_start = 0 # idx of the last word's first character
    
    for i in range(n):
        # checking if this is the last character in the line
        if cur_len == ch_per_line - 1:
            # if last char in the line is a space
            if proverb[i] == ' ':
                cur_line_interval[1] = i
                line_intervals.append(cur_line_interval)
                cur_line_interval = [i + 1, None]
                prev_word_start = i + 1
                cur_len = 0
            else:
                # if last char in the line is the end of the whole proverb
                if i == (n - 1):
                    cur_line_interval[1] = i + 1
                    line_intervals.append(cur_line_interval)
                # if the last char in the line is the end of a word (and not end of proverb)
                elif proverb[i + 1] == ' ':
                    cur_line_interval[1] = i + 1
                    line_intervals.append(cur_line_interval)
                    cur_line_interval = [i + 2, None]
                    cur_len = 0
                # if we are currently breaking up a word
                else:
                    cur_line_interval[1] = prev_word_start - 1
                    line_intervals.append(cur_line_interval)
                    cur_line_interval = [prev_word_start, None]
                    cur_len = i - prev_word_start + 1
        # end of proverb
        elif i == (n-1):
            cur_line_interval[1] = n
            line_intervals.append(cur_line_interval)
        # continue through letters
        else:
            if proverb[i] == ' ':
                prev_word_start = i + 1
            cur_len += 1
            
    return line_intervals

In [165]:
def draw_boxes_triangles(c, lines, line_offsets, letter_reveals):
    for line_num, line in enumerate(lines):
        for i, ch in enumerate(line):
            if ch != ' ':
                # bottom right x-coord of box
                box_x = x_center + ((i - len(line)/2)* BOX_LEN)
                # bottom right y-coord of box
                box_y= PUZZLE_TOP_Y - (line_num * box_vertical_spacing)
                # drawing box for letter
                c.rect(box_x, box_y, BOX_LEN, BOX_LEN, stroke=1, fill=0)
                
                # setting font to letter
                c.setFont(BODY_FONT, LETTER_FONT_SIZE)
                
                # drawing in letter if needs to be revealed
                ch_num = line_offsets[line_num] + i # idx of char in original proverb
                if (ch_num + 1) in letter_reveals:
                    ch_width = stringWidth(ch, BODY_FONT, LETTER_FONT_SIZE)
                    ch_x = box_x + (BOX_LEN/2) - (ch_width/2)
                    ch_y = box_y + (BOX_LEN - LETTER_FONT_SIZE) * 1.00 # 1.00 is tunable
                
                    # drawing letter in box
                    c.drawString(ch_x, ch_y, ch)  
                
                # setting font back to body
                c.setFont(BODY_FONT_BOLD, BODY_FONT_SIZE)

                 # bottom vertex
                bottom_vx_x, bottom_vx_y = box_x + (BOX_LEN/2), box_y + BOX_LEN
                # top left vertex
                top_l_vx_x, top_l_vx_y = box_x, box_y + (BOX_LEN + TRIANGLE_HEIGHT)
                # top right vertex
                top_r_vx_x, top_r_vx_y = box_x + BOX_LEN, box_y + (BOX_LEN + TRIANGLE_HEIGHT)

                # draw lines connecting vertices
                c.line(bottom_vx_x, bottom_vx_y, top_l_vx_x, top_l_vx_y)
                c.line(bottom_vx_x, bottom_vx_y, top_r_vx_x, top_r_vx_y)
                c.line(top_l_vx_x, top_l_vx_y, top_r_vx_x, top_r_vx_y)

                # finding center of triangle
                tr_middle_x, tr_middle_y = bottom_vx_x, (top_l_vx_y + bottom_vx_y)/2

                # finding width of text for centering
                num = str(CHAR_TO_NUM[ch])
                text_width = stringWidth(num, BODY_FONT_BOLD, BODY_FONT_SIZE)

                # finding bottom left of text
                text_x = tr_middle_x - (text_width/2)
                text_y = tr_middle_y - (BODY_FONT_SIZE/6) # approximate, might need some fiddling

                # drawing in letter
                c.drawString(text_x, text_y, num)

In [166]:
def draw_circles(c):
    # compute number of circles
    num_circles = len(CIRCLES)
    
    for i, text in enumerate(CIRCLES):
        # offset to center circles horizontally
        offset = 0.5 * ((CIRCLE_RADIUS * 2) * (num_circles - 1) + (num_circles - 1) * CIRCLE_SPACE)

        # circle center coords
        circle_x = (WIDTH/2) + (i * (2 * CIRCLE_RADIUS + CIRCLE_SPACE)) - offset
        circle_y = CIRCLE_Y

        # draw circle
        c.circle(circle_x, circle_y, CIRCLE_RADIUS, stroke=1, fill=0)

        # finding width of text for centering
        text_width = stringWidth(text, BODY_FONT_BOLD, BODY_FONT_SIZE)
        text_x = circle_x - text_width/2
        text_y = circle_y + BODY_FONT_SIZE * 0.20 # 0.40 is tunable

        # writing text
        num = str(i)
        c.drawString(text_x, text_y, text)

        # finding width of number for centering
        num_width = stringWidth(num, BODY_FONT_BOLD, BODY_FONT_SIZE)
        num_x = circle_x - num_width/2
        num_y = circle_y - BODY_FONT_SIZE

        # writing number
        c.drawString(num_x, num_y, num)

In [167]:
def log_proverb(proverb):
    # logging the proverb and it's date and time in a csv file
    file_exists = os.path.isfile(LOG_FILENAME)
    with open(LOG_FILENAME, 'a', newline='') as file:
        writer = csv.writer(file)
        # write header if the file doesn't exist
        if not file_exists:
            writer.writerow(['Date and Time', 'Proverb'])
        date_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S") # Year-Month-Day Hour:Minute:Second
        writer.writerow([date_time, proverb])

In [168]:
def setup_file_structure():
    # create proverb_puzzles folder if it doesn't exist
    directory = os.path.dirname(FOLDER)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [169]:
def clean_proverb(proverb):
    # removing all special character
    proverb = re.sub(PATTERN, '', proverb)
    
    # cleaning up spaces and making all uppercase
    proverb = list(proverb.upper().strip())
    
    # length of cleaned proverb
    n = len(proverb)
    
    return proverb, n

In [170]:
def draw_footer(c, clue, prev, num_lines):
    # properly format clue  
    clue_label = "Clue: "
    clue = clue.strip()
    is_clue = len(clue)
    
    # checking if there is a clue
    if is_clue:

        # setting font to bold for "Clue: "
        c.setFont(BODY_FONT_BOLD, FOOTER_FONT_SIZE)

        # calculating "Clue: " location
        clue_label_width = stringWidth(clue_label, BODY_FONT_BOLD, FOOTER_FONT_SIZE)
        clue_label_x = MARGIN + 0.50 * (WIDTH - (2 * MARGIN) - (ch_per_line * BOX_LEN))
        clue_label_y = PUZZLE_TOP_Y - ((num_lines - 1) * (TRIANGLE_HEIGHT + BOX_LEN) 
                                 + ((num_lines - 1) * LINE_SPACING)
                                 + FOOTER_FONT_SIZE
                                 + LINE_SPACING/2)

        # drawing in clue label
        c.drawString(clue_label_x, clue_label_y, clue_label)

        # setting font for Clue
        c.setFont(BODY_FONT_BOLD_OBLIQUE, FOOTER_FONT_SIZE)

        # calculating clue text location
        clue_x = clue_label_x + clue_label_width # 0.8 is tunable
        clue_y = clue_label_y

        # drawing in clue text
        c.drawString(clue_x, clue_y, clue)
    
    prev = prev.strip().upper()
    is_prev = len(prev)
    
    if is_prev:
        # writing yesterday's answer
        pt1, pt2, pt3 = "Yesterday's ", "P.A.S.S.", "WORDS: " # normal, bold, normal
        pt1_width = stringWidth(pt1, BODY_FONT, FOOTER_FONT_SIZE)
        pt2_width = stringWidth(pt2, BODY_FONT_BOLD, FOOTER_FONT_SIZE)
        pt3_width = stringWidth(pt3, BODY_FONT, FOOTER_FONT_SIZE)
        prev_width = stringWidth(prev, BODY_FONT_OBLIQUE, FOOTER_FONT_SIZE)
        
        # calculating widths for all text
        label_width = pt1_width + pt2_width + pt3_width
        total_width = label_width + prev_width
        max_width = WIDTH - MARGIN * 2
        
        # maximum of two lines for this section (previous day's proverb)
        split_point = None
        if total_width > max_width:
            word_breaks = [i for i, ch in enumerate(prev) if ch == ' ']
            
            # iterating backward through word breaks to see where to split new line
            for break_idx in word_breaks[::-1]:
                new_width = stringWidth(prev[:break_idx], BODY_FONT_OBLIQUE, FOOTER_FONT_SIZE)
                # width of second line, with leading space removed
                remaining_width = stringWidth(prev[break_idx + 1:], BODY_FONT_OBLIQUE, FOOTER_FONT_SIZE)
                line1_width = label_width + new_width
                
                # if this new width fits, then split here, else continue
                if line1_width <= max_width:
                    split_point = break_idx
                    line2 = prev[split_point:].strip().upper()
                    break
        
        # different offsets based on if line was split or not
        if split_point == None:
            offset = total_width/2
        else:
            offset = line1_width/2
            
        # calculating and displaying the first line of the prev day's word
        pt1_x = (WIDTH/2) - offset
        pt2_x = pt1_x + pt1_width
        pt3_x = pt2_x + pt2_width
        prev_x = pt3_x + pt3_width

        if is_clue:
            pt1_y = pt2_y = pt3_y = prev_y = clue_y - FOOTER_SPACING
        else:
            pt1_y = pt2_y = pt3_y = prev_y = PUZZLE_TOP_Y - ((num_lines - 1) * (TRIANGLE_HEIGHT + BOX_LEN) 
                                                             + ((num_lines - 1) * LINE_SPACING)
                                                             + FOOTER_FONT_SIZE
                                                             + FOOTER_SPACING)

        # setting font for pt1 ("Yesterday's ")
        c.setFont(BODY_FONT, FOOTER_FONT_SIZE)
        c.drawString(pt1_x, pt1_y, pt1)

        # setting font for pt2 ("P.A.S.S")
        c.setFont(BODY_FONT_BOLD, FOOTER_FONT_SIZE)
        c.drawString(pt2_x, pt2_y, pt2)

        # setting font for pt3 ("WORDS: ")
        c.setFont(BODY_FONT, FOOTER_FONT_SIZE)
        c.drawString(pt3_x, pt3_y, pt3)

        # setting font for prev (last proverb)
        c.setFont(BODY_FONT_OBLIQUE, FOOTER_FONT_SIZE)
        
        # draw depending on 
        if split_point == None:
            c.drawString(prev_x, prev_y, prev)
        else:
            c.drawString(prev_x, prev_y, prev[:split_point])
            
        # draw second line if necessary
        if split_point:
            offset2 = remaining_width/2
            line2_x = (WIDTH/2) - offset2
            line2_y = pt1_y - FOOTER_SPACING
            
            c.drawString(line2_x, line2_y, line2)
            

In [171]:
def collapse_spaces(arr):
    for i, s in enumerate(arr):
        arr[i] = re.sub(r'\s+', ' ', s).strip()
    return arr

In [184]:
def draw_description(c):
    line1 = ["Determine the ", "P", "roverb, ", "A", "dage, ", "S", 
             "aying, or ", "S", "aw by converting the triangled numbers"]
    bold = [False, True, False, True, False, True, False, True, False]
    line_2 = "to letters by connecting the boxes using the corresponding circular alphanumeric key."
    
    line1_widths = []
    for i, segment in enumerate(line1):
        if bold[i]:
            line1_widths.append(stringWidth(segment, BODY_FONT_BOLD, DESCRIPTION_FONT_SIZE))
        else:
            line1_widths.append(stringWidth(segment, BODY_FONT, DESCRIPTION_FONT_SIZE))
    
    line1_width = sum(line1_widths)
    line2_width = stringWidth(line_2, BODY_FONT, DESCRIPTION_FONT_SIZE)
    
    line1_x = (WIDTH/2) - (line1_width/2)
    line2_x = (WIDTH/2) - (line2_width/2)
    
    line2_y = CIRCLE_Y + CIRCLE_RADIUS + LINE_SPACING/2 # denom is tunable
    line1_y = line2_y + BODY_FONT_SIZE + LINE_SPACING/3 # denom is tunable
    
    # setting font to body
    c.setFont(BODY_FONT, DESCRIPTION_FONT_SIZE)
    
    # draw all of line 1
    for i, segment in enumerate(line1):
        if bold[i]:
            # setting font to bold
            c.setFont(BODY_FONT_BOLD, DESCRIPTION_FONT_SIZE)
        else:
            # setting font to normal
            c.setFont(BODY_FONT, DESCRIPTION_FONT_SIZE)
            
        c.drawString(sum(line1_widths[:i]) + line1_x, line1_y, segment)
    
    # draw line 2
    c.drawString(line2_x, line2_y, line_2)

In [185]:
def generate_puzzle(filename, proverb, clue='', prev='', reveal_letters=[]):
    
    # initialize file structure if not existing
    setup_file_structure()
    
    # create canvas in correct folder
    filepath = FOLDER + filename
    c = canvas.Canvas(filepath, pagesize=letter)

    # collapses all spaces into single space
    proverb, clue, prev = collapse_spaces([proverb, clue, prev])
    # removing special characters and extra spaces, uppercasing
    proverb, n = clean_proverb(proverb)
    
    # checking that proverb exists
    if n == 0:
        raise ValueError("Proverb must be of length at least 1.")
    
    # setting font for body
    c.setFont(BODY_FONT_BOLD, BODY_FONT_SIZE)
    
    # intervals of the form (start, end) for each generated line
    line_intervals = separate_lines(proverb, n) 
            
    # splitting up proverb into lines
    lines = []
    line_offsets = [] # to find index of each char in original proverb (without removing trailing spaces) 
    num_lines = 0
    for i,j in line_intervals:
        lines.append(proverb[i:j])
        line_offsets.append(i)
        num_lines += 1
    
    # draw a rectangle and triangle for each letter
    draw_boxes_triangles(c, lines, line_offsets, reveal_letters)

    # draw predefined circles
    draw_circles(c)
    
#     # temp to draw margin
#     # REMOVE LATER
    x_width, y_height = WIDTH -  (2 * MARGIN), HEIGHT - (2 * MARGIN)
    c.rect(MARGIN, MARGIN, x_width, y_height, stroke=1, fill=0)
    
    # drawing top description
    draw_description(c)
    
    # drawing footer (clue + prev answer)
    draw_footer(c, clue, prev, num_lines)
    
    # add proverb and date/time to CSV file log.csv
    log_proverb(''.join(proverb))
    
    c.showPage()
    c.save()

In [186]:
generate_puzzle("test2.pdf", "hi I am texting this", clue="Testing the clue", reveal_letters=[1, 2, 5, 7, 25], prev='KLMNOPQRS TUVWXYZ')